<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/wordEmbedding/WordsEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Kingston/ZSL-v2/

Mounted at /content/drive
/content/drive/MyDrive/Kingston/ZSL-v2


In [9]:
!pip install transformers gensim wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=8573dfff407f257f6688d172ca826819d1e7eb89847b1970c7473e2fd877a119
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
!git pull

remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/Taedriel/ZSL-v2
   1d32dea..d684304  wordEmbedding  -> origin/wordEmbedding
   3603bd6..0a4103b  googleImageFSL -> origin/googleImageFSL
Updating 1d32dea..d684304
Fast-forward
 WordsEmbeddings.ipynb | 101 ++++++++++++++++++++++++++++++++------------------
 1 file changed, 64 insertions(+), 37 deletions(-)


In [31]:
from converter import *
model = BERTModel(["king", "queen", "man", " woman"])

# model.importTagList("en-basic")
model.convert()
model.export("en-basic-Embedded.csv")

# model.computeCoSim()
# model.simBetween("cat", "dog")

man = model.getEmbeddingOf("man")[1]
woman = model.getEmbeddingOf("woman")[1]

king = model.getEmbeddingOf("king")[1]

totest = king.sub(man).add(woman)
model.getNearestEmbeddingOf(totest, 3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('woman', tensor(0.7539)),
 ('queen', tensor(0.7833)),
 ('king', tensor(0.8733))]

In [14]:
# %cd dataset/
# !wget https://www.dropbox.com/s/kguufyc2xcdi8yk/lexvec.enwiki%2Bnewscrawl.300d.W.pos.vectors.gz?dl=1
# !mv lexvec.enwiki+newscrawl.300d.W.pos.vectors.gz\?dl\=1 lexvec_vectors.gz
# !gunzip lexvec_vectors.gz
# %cd ..

/content/drive/MyDrive/Kingston/ZSL-v2


In [26]:
from gensim.models import KeyedVectors
import gensim.downloader

# model = gensim.downloader.load('word2vec-google-news-300')
model = KeyedVectors.load_word2vec_format('dataset/lexvec_vectors', binary=False)

vocab = model.vocab.keys()

INFO:gensim.models.utils_any2vec:loading projection weights from dataset/lexvec_vectors
INFO:gensim.models.utils_any2vec:loaded (368999, 300) matrix from dataset/lexvec_vectors


In [30]:
from numpy import dot
from numpy.linalg import norm


sentence = ["boy", "girl", "man", "woman"]
vectors = []

for w in sentence:
    if w in vocab:
        vectors.append((w, model[w]))
    else:
        print("Word {} not in vocab".format(w))
        vectors.append([0])

cosine_sim = lambda a, b : dot(a, b)/(norm(a)*norm(b))

king = [i[1] for i in vectors if i[0] == "boy"][0]
man = [i[1] for i in vectors if i[0] == "man"][0]
woman = [i[1] for i in vectors if i[0] == "woman"][0]
queen = [i[1] for i in vectors if i[0] == "girl"][0]


totest = king - man + woman
result = []
for i in vectors:
    sim = cosine_sim(i[1], totest)
    result.append((i[0], sim))

result.sort(key = lambda t : t[1])
print(result)

print(cosine_sim(king, man))
print(cosine_sim(queen, woman))

# [('man', 0.033279095), ('woman', 0.37786838), ('Queen', 0.55495286), ('King', 0.7780154)]
# 0.15777647
# 0.20171621

[('man', 0.43875915), ('woman', 0.71617275), ('girl', 0.85282505), ('boy', 0.8802507)]
0.6430704
0.76060945
